In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA   

Read the NSBCD file

In [2]:
file = open('NSBCD_train_1.csv', 'r')
line = file.readline()
NUM_FEATURES = len(line.split(",")) - 2
print(NUM_FEATURES)
survival_times = []
X = []
maximum_ob_time = 0
while line != "":
    segmentations = line.split(",")
    if segmentations[-1] == 1:
        continue
    else:
        survival_times.append(int(segmentations[-2]))
        entries = np.array(segmentations[:-2])
        for entry in entries:
            X.append(float(entry))
    line = file.readline()

NUM_TRAIN = int(len(X) / NUM_FEATURES)

file = open('NSBCD_test_1.csv', 'r')
line = file.readline()
while line != "":
    segmentations = line.split(",")
    if segmentations[-1] == 1:
        continue
    else:
        survival_times.append(int(segmentations[-2]))
        entries = np.array(segmentations[:-2])
        for entry in entries:
            X.append(float(entry))
    line = file.readline()

X = np.reshape(X, (-1, NUM_FEATURES))
# X = X[:,1:]
NUM_EMPLOYEES = X.shape[0]

# print(X.shape)
# pca=PCA(n_components=115)  
# X =pca.fit_transform(X)

NUM_VAL = NUM_EMPLOYEES - NUM_TRAIN

maximum_ob_time = max(survival_times)
Y = np.ones((NUM_EMPLOYEES, maximum_ob_time))
# print(X)
Mi = np.max(X, axis = 0)
mi = np.min(X, axis = 0)
mi[0] = 0
X = 2 * (X - mi) / (Mi - mi) -1
# print(X)
for i in range(NUM_EMPLOYEES):
    Y[i, survival_times[i] - 1:] = -1

# print(X[0])
print(X[0])

550
[ 1.00000000e+00 -9.60593451e-02 -4.66802236e-01  1.00000000e+00
 -5.21411066e-01  5.20446097e-02  2.62575263e-01 -1.91711813e-01
  8.10502283e-01 -2.74944568e-01  5.00000000e-01  3.01558466e-01
  4.90256791e-01 -7.24814942e-01  5.16709512e-01 -3.61038961e-01
 -8.92988930e-01 -9.05824323e-02 -3.50348028e-01  2.24971841e-01
  2.40131579e-01 -1.88786987e-01 -3.52154532e-01 -3.65269461e-01
  3.72718585e-01  1.89333333e-01  8.04051225e-01  2.43396287e-01
 -3.34261838e-02  3.22695418e-01 -9.03171953e-01 -1.41390434e-01
 -8.54951185e-01 -7.48560461e-02  1.64738209e-01 -4.96074770e-01
 -8.15219887e-02  1.90697674e-01  1.00000000e+00  6.15570400e-01
  3.74171458e-01  3.51285190e-01  7.85287405e-01  3.74811422e-01
 -5.95947557e-03  2.54889996e-01 -2.68535935e-02 -4.84937207e-01
  5.08368201e-01  1.33280715e-01 -2.38261930e-01 -1.00000000e+00
  3.36971441e-01 -9.03377557e-02 -1.40186561e-01  7.74358974e-01
 -8.52713178e-02  6.04026846e-02  3.04682971e-01 -7.91975143e-02
 -9.14407989e-01 -1.9

In [3]:
BANDWIDTH = 10
max_iteration = 10000
weight_decay = 0.01
d = 10

In [4]:
X_train = X[:NUM_TRAIN]
X_val = X[NUM_TRAIN:]
Y_train = Y[:NUM_TRAIN]
Y_val = Y[NUM_TRAIN:]

In [5]:
square = np.sum(X ** 2, axis=1)
column_vec = square[:, np.newaxis]
row_vec = square[np.newaxis, :]

# Gaussian_Kernel = np.exp(-1 * (-2 * X.dot(X.T) + column_vec + row_vec) / (2 * BANDWIDTH ** 2)) # the Gaussian Kernel
Gaussian_Kernel = ((X.dot(X.T)) + 1) ** d

# norm_matrix = (square[:,np.newaxis].dot(square[np.newaxis,:])) ** 0.5
# Gaussian_Kernel = X.dot(X.T) # / norm_matrix

Gaussian_Kernel_Train = Gaussian_Kernel[:NUM_TRAIN,:NUM_TRAIN]
Gaussian_Kernel_Val = Gaussian_Kernel[NUM_TRAIN:,:NUM_TRAIN]

In [6]:
print(Gaussian_Kernel_Val[0])

[2.12942445e+14 3.03579253e+14 2.93013829e+14 3.72827678e+13
 6.35127962e+13 3.77920965e+13 6.13656048e+14 7.01037323e+13
 5.28823247e+14 1.36485871e+15 4.27146273e+14 2.73594804e+14
 7.46884646e+13 1.92724294e+15 2.85247224e+13 1.00205277e+15
 1.63944178e+14 4.32873619e+15 1.09611702e+15 1.91832596e+15
 1.47572019e+13 3.53449368e+14 1.42937117e+15 8.66081033e+12
 4.31186884e+14 9.45104707e+11 3.73241223e+13 1.34713973e+15
 1.58994537e+14 4.41700160e+13 1.36154938e+15 1.09673718e+15
 6.68813534e+11 1.91223914e+12 1.17706618e+15 8.71205643e+13
 7.71523572e+14 7.87269120e+12 2.10243033e+14 7.35166463e+14
 1.60673055e+13 5.40244593e+14 6.88966601e+14 7.81273071e+14
 1.83051730e+15 2.13979017e+14 4.11615428e+13 7.23215276e+12
 1.07454350e+13 3.22987893e+12 2.72517829e+14 1.28547055e+10
 9.67876170e+11 1.81339906e+14 3.94583153e+14 6.74475395e+13
 2.49133986e+12 5.33127046e+13 2.46877719e+13 9.10385926e+13
 6.16250448e+12 2.17609492e+14 2.50498541e+13 7.30487718e+13
 1.65133431e+14 3.040328

In [7]:
alpha = np.zeros_like(Y_train) 
# alpha = np.ones_like(Y_train)
predict_age = np.zeros(NUM_EMPLOYEES) 
val_loss_list = []
for t in range(1, max_iteration+1):
    '''
    index = int(np.floor(np.random.rand()*NUM_TRAIN)) # randomly select one employee
    haty_index = (Gaussian_Kernel_Train[index,:][np.newaxis,:].dot(alpha * Y_train)).reshape(-1) # calculate the predicted y vector
    haty_index /= t * weight_decay
    '''
    indices = np.random.rand(5,1) * NUM_TRAIN
    indices = indices.astype(int).reshape(-1)
    print(indices)
    print(Gaussian_Kernel_Train[indices].shape)
    haty_indices = (Gaussian_Kernel_Train[indices].dot(alpha * Y_train)) 
    print()

    ## update alpha

    mask = Y_train[indices]*haty_indices < 1
    alpha[indices] += mask

    if t%1000 == 0 or t == 1:
        print("the iteration is:", t)
        hatY_train = np.zeros_like(Y_train)
        hatY_train += Gaussian_Kernel_Train.dot(alpha*Y_train)
        hatY_train /= t * weight_decay
        
        hatY_val = np.zeros_like(Y_val)
        hatY_val += Gaussian_Kernel_Val.dot(alpha*Y_train)
        hatY_val /= t * weight_decay
        
        for i in range(NUM_TRAIN):
            # print(np.where(hatY_train[i,:] < 0)[0])
             predict_age[i] = np.where(hatY_train[i,:] < 0)[0][0] + 1
            # predict_age[i] = np.where(hatY_train[i,:] < 0)[0][0] + 1
        for i in range(NUM_TRAIN,NUM_EMPLOYEES):
            predict_age[i] = np.where(hatY_val[i-NUM_TRAIN,:] < 0)[0][0] + 1
        
        acc_train = np.sum(Y_train * hatY_train > 0) / (NUM_TRAIN * maximum_ob_time)
        acc_val = np.sum(Y_val * hatY_val > 0) / (NUM_VAL * maximum_ob_time)
        print("the train accuracy is:", acc_train)
        print("the val accuracy is:", acc_val)
        
        '''
     
        val_result = hatY_val * Y[NUM_TRAIN:]
        mask = val_result < 0
        
        val_loss = np.sum(val_result < 0) - np.sum(val_result[mask])
        if t == 1 or v/al_loss < loss_list[-1]:
            val_loss_list.append(val_loss)
        
        '''
        
        useful_pair_tr = 0.0
        denominator_tr = NUM_TRAIN*(NUM_TRAIN-1)/2
        
        for i in range(NUM_TRAIN):
            for j in range(i+1, NUM_TRAIN):
                if (survival_times[i]-survival_times[j])*(predict_age[i]-predict_age[j])> 0: # useful pair
                    useful_pair_tr += 1
                if  survival_times[i] == survival_times[j]: # two employees whose lifetime are identical
                    denominator_tr -= 1
        
        useful_pair_val = 0.0
        denominator_val = NUM_VAL*(NUM_VAL-1)/2
        
        for i in range(NUM_TRAIN, NUM_EMPLOYEES):
            for j in range(i+1, NUM_EMPLOYEES):
                if (survival_times[i]-survival_times[j])*(predict_age[i]-predict_age[j])> 0: # useful pair
                    useful_pair_val += 1
                if  survival_times[i] == survival_times[j]: # two employees whose lifetime are identical
                    denominator_val -= 1
        
        c_index_tr = useful_pair_tr / denominator_tr
        c_index_val = useful_pair_val / denominator_val
        print("the c-index for TRAINING is,",c_index_tr)
        print("the c-index for VALIDATION is,",c_index_val)
        
print(np.sum(predict_age[NUM_TRAIN:] == survival_times[NUM_TRAIN:])/(NUM_VAL))

[11 59 72 67 48]
(5, 76)


ValueError: operands could not be broadcast together with shapes (5,188) (940,) 

In [ ]:
x_axis = np.arange(maximum_ob_time)+1
y_train_gt_axis = np.zeros(maximum_ob_time)
y_val_gt_axis = np.zeros(maximum_ob_time)
y_train_predict_axis = np.zeros(maximum_ob_time)
y_val_predict_axis = np.zeros(maximum_ob_time)

for i in range(maximum_ob_time):
    y_train_gt_axis[i] = np.sum(survival_times[:NUM_TRAIN] == x_axis[i])
    y_val_gt_axis[i] = np.sum(survival_times[NUM_TRAIN:] == x_axis[i])
    y_train_predict_axis[i] = np.sum(predict_age[:NUM_TRAIN] == x_axis[i])
    y_val_predict_axis[i] = np.sum(predict_age[NUM_TRAIN:] == x_axis[i])
# plt.plot(x_axis, y_train_gt_axis, x_axis, y_train_predict_axis)
plt.plot(x_axis, y_val_gt_axis, x_axis, y_val_predict_axis)

In [ ]:
hatY_val += Gaussian_Kernel_Val.dot(alpha*Y_train)
hatY_val /= t * weight_decay
print(Gaussian_Kernel_Val[0])
print(hatY_val[0])
print(Y_val[0])